In [1]:
%matplotlib widget

In [2]:
import ipywidgets as widgets
from IPython.display import display
from geometry_msgs.msg import Point
from assignment_2_2023.msg import Target
from std_msgs.msg import Empty

import matplotlib.pyplot as plt
import rospy
from tf.transformations import quaternion_matrix
import numpy as np
from assignment_2_2023.msg import Posvel
from matplotlib.animation import FuncAnimation

In [3]:
# Initialize the ROS node
rospy.init_node('odom_visualizer_node')
goal_pub = rospy.Publisher('/goal', Target, queue_size=10)
cancel_pub = rospy.Publisher('/cancel_goal', Empty, queue_size=10)

In [4]:
# Publishers


# Function to publish the target coordinates
def publish_goal(x, y):
    msg_target = Target()
    msg_target.x = x
    msg_target.y = y
    goal_pub.publish(msg_target)
    print(f"Goal published: x = {x}, y = {y}")
    return

# Function to cancel the goal
def cancel_goal(b):
    cancel_pub.publish(Empty())
    print("Goal cancelled")

# Widget callbacks
def on_send_goal_clicked(b):
    try:
        x = float(x_text.value)
        y = float(y_text.value)
        publish_goal(x, y)
    except ValueError as e:
        print(f"Error: {e}")

def on_cancel_goal_clicked(b):
    cancel_goal(b)

# Create widgets
x_text = widgets.FloatText(value=0.0, description='X:', style={'description_width': 'initial'})
y_text = widgets.FloatText(value=0.0, description='Y:', style={'description_width': 'initial'})
send_goal_button = widgets.Button(description='Send Goal', button_style='success', tooltip='Send new XY coordinates as goal')
cancel_goal_button = widgets.Button(description='Cancel Goal', button_style='danger', tooltip='Cancel current goal')

# Assign widget callbacks
send_goal_button.on_click(on_send_goal_clicked)
cancel_goal_button.on_click(on_cancel_goal_clicked)




In [4]:
# Function to setup floating point text widgets
def setup_floatText(description):
    return widgets.FloatText(value=0.0, description=description, disabled=False)

# Function to setup button widgets
def setup_button(name, icon, style=''):
    return widgets.Button(description=name, button_style=style, icon=icon, tooltip=name)

# Callback for sending the target to ROS
def send_target(click):
    target_msg = Target()
    target_msg.x = x_widget.value
    target_msg.y = y_widget.value
    pub_goal.publish(target_msg)
    print(f"Goal published: x = {target_msg.x}, y = {target_msg.y}")

# Callback for canceling the goal in ROS
def cancel_goal(b):
    pub_cancel_goal.publish(Empty())
    print("Goal cancelled")

# Set up widgets
x_widget = setup_floatText('X:')
y_widget = setup_floatText('Y:')
b_set_target = setup_button('Send New Target', 'paper-plane', 'success')
b_cancel_goal = setup_button('Cancel Current Goal', 'stop-circle', 'danger')

# Assign callbacks to buttons
b_set_target.on_click(send_target)
b_cancel_goal.on_click(cancel_goal)

# Layout the widgets using IPython widgets layout
from ipywidgets import TwoByTwoLayout
widgets_layout = TwoByTwoLayout(top_left=x_widget,
                                top_right=b_set_target,
                                bottom_left=y_widget,
                                bottom_right=b_cancel_goal)

display(widgets_layout)


TwoByTwoLayout(children=(FloatText(value=0.0, description='X:', layout=Layout(grid_area='top-left')), Button(b…

In [5]:


class Visualiser:
    def __init__(self):
        # Set up the plot environment
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro', label='Robot Position')  # 'ro' for red dots
        self.goal_marker, = plt.plot([], [], 'gx', markersize=10, label='Goal Position')  # 'gs' for green square
        self.x_data, self.y_data = [] , []
        self.goal_x, self.goal_y = None, None

    def plot_init(self):
        # Set the limits of the plot
        self.ax.set_xlim(0, 10)  # Adjust based on your robot's operating area
        self.ax.set_ylim(0, 10)
        self.ax.legend()
        return self.ln, self.goal_marker
        
    def odom_callback(self, msg):
        # Append the position data received from the Posvel message
        self.y_data.append(msg.y)
        self.x_data.append(msg.x)

    def goal_callback(self, msg):
        # Update the goal position
        self.goal_x = msg.x
        self.goal_y = msg.y
        # Reset the x_data and y_data when a new goal is received
        self.x_data, self.y_data = [], []

    def update_plot(self, frame):
        # Update the robot position plot
        self.ln.set_data(self.x_data, self.y_data)
        # Update the goal position plot if it's set
        if self.goal_x is not None and self.goal_y is not None:
            self.goal_marker.set_data([self.goal_x], [self.goal_y])
        return self.ln, self.goal_marker
        


# Create an instance of the Visualiser class
vis = Visualiser()

# Subscribe to the '/pos' topic for robot position and '/goal' for goal position
sub_pos = rospy.Subscriber('/pos', Posvel, vis.odom_callback)
sub_goal = rospy.Subscriber('/goal', Target, vis.goal_callback)

# Set up the animation
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)

# Display the plot
plt.show(block=True)

# Display widgets
display(x_text, y_text, send_goal_button, cancel_goal_button)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'x_text' is not defined

In [6]:
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)